# 📊 IU X-ray Report Evaluation Dashboard

This notebook visualizes the quality of generated radiology reports using:

- **BLEU** and **ROUGE** score distributions
- Report refinement improvement (initial → refined)
- Co-occurrence heatmap of predicted labels


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

# Load inference results
df_results = pd.read_csv("/kaggle/working/final_inference_results.csv")


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(df_results['bleu_init'], bins=30, alpha=0.5, label='Initial')
plt.hist(df_results['bleu_ref'], bins=30, alpha=0.5, label='Refined')
plt.title("BLEU Score Distribution")
plt.xlabel("BLEU Score"); plt.ylabel("Count"); plt.legend()

plt.subplot(1, 2, 2)
plt.hist(df_results['rouge1_init'], bins=30, alpha=0.5, label='Initial')
plt.hist(df_results['rouge1_ref'], bins=30, alpha=0.5, label='Refined')
plt.title("ROUGE-1 F1 Distribution")
plt.xlabel("ROUGE Score"); plt.ylabel("Count"); plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
df_results['bleu_improvement'] = df_results['bleu_ref'] - df_results['bleu_init']
df_results['rouge_improvement'] = df_results['rouge1_ref'] - df_results['rouge1_init']

plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
plt.hist(df_results['bleu_improvement'], bins=30, color='orange')
plt.axvline(df_results['bleu_improvement'].mean(), color='red', linestyle='--')
plt.title("BLEU Score Improvement")

plt.subplot(1, 2, 2)
plt.hist(df_results['rouge_improvement'], bins=30, color='green')
plt.axvline(df_results['rouge_improvement'].mean(), color='red', linestyle='--')
plt.title("ROUGE-1 F1 Improvement")

plt.tight_layout()
plt.show()


In [ ]:
df_results['label_list'] = df_results['predicted_labels'].fillna('').apply(lambda x: x.split(';') if x else [])
mlb = MultiLabelBinarizer()
binary_matrix = mlb.fit_transform(df_results['label_list'])
co_matrix = pd.DataFrame(binary_matrix.T @ binary_matrix, index=mlb.classes_, columns=mlb.classes_)

plt.figure(figsize=(10, 8))
sns.heatmap(co_matrix, annot=True, fmt='d', cmap='YlGnBu')
plt.title("Predicted Label Co-occurrence Heatmap")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
